In [ ]:
import sys; sys.path.append('..')
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
r = pickle.load(open('../data/bad_rod.pkl', 'rb'))
fixedVars = [0, 1, 2, 77, 150, 151, 152]
view = linkage_vis.LinkageViewer(r, width=768, height=768)
view.show()

In [ ]:
from contextlib import contextmanager
import sys, os
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try: yield
        finally: sys.stdout = old_stdout

In [ ]:
view.exportHTML('view.html')

In [ ]:
elastic_rods.compute_equilibrium(r, verbose=True, fixedVars=fixedVars, niter=100)
view.update()

In [ ]:
with suppress_stdout():
    t = time.time()
    nit = elastic_rods.compute_equilibrium(r, verbose=False, fixedVars=fixedVars, niter=10000)
    simTime = time.time() - t
print('Sim time: ', simTime)
print('iterations: ', nit)
view.update()

In [ ]:
def energyAt(rod, dof, etype = elastic_rods.EnergyType.Full):
    prevDoF = rod.getDoFs()
    r.setDoFs(dof)
    energy = rod.energy(etype)
    r.setDoFs(prevDoF)
    return energy
from enum import Enum
class DiffScheme(Enum):
    FORWARD = 1
    BACKWARD = 2
    CENTERED = 3
def fd_gradient(r, stepSize, etype=elastic_rods.EnergyType.Full, scheme=DiffScheme.CENTERED):
    def step(i):
        d = np.zeros(r.numDoF())
        d[i] = stepSize
        return d
    
    if (scheme == DiffScheme.CENTERED): plus, minus, den = (lambda s: r.getDoFs() + s, lambda s: r.getDoFs() - s, 2 * stepSize)
    if (scheme == DiffScheme.BACKWARD): plus, minus, den = (lambda s: r.getDoFs()    , lambda s: r.getDoFs() - s,     stepSize)
    if (scheme == DiffScheme.FORWARD ): plus, minus, den = (lambda s: r.getDoFs() + s, lambda s: r.getDoFs()    ,     stepSize)
    return np.array([energyAt(r, plus(step(i)), etype) - energyAt(r, minus(step(i)), etype) for i in range(r.numDoF())]) / den
def fd_gradient_test(r, stepSize, direction, etype=elastic_rods.EnergyType.Full):
    grad = r.gradient(False, etype)
    return [(energyAt(r, r.getDoFs() + stepSize * direction, etype) - energyAt(r, r.getDoFs() - stepSize * direction, etype)) / (2 * stepSize), np.dot(direction, grad)]
def plotEnergy(rod, direction, steps):
    restDoF = rod.getDoFs()
    plt.plot(steps, [energyAt(rod, restDoF + s * direction) for s in steps])

In [ ]:
Htrip = r.hessian()
Htrip.reflectUpperTriangle()
for i in range(Htrip.m):
    Htrip.addNZ(i, i, 0.000000167)
H = csc_matrix(Htrip.compressedColumn())
Hspsd = sparse_matrices.SPSDSystem(Htrip)
Hspsd.fixVariables(fixedVars, [0 for i in fixedVars])

g = r.gradient()
gzero = g.copy()
gzero[fixedVars] = 0
direction = Hspsd.solve(-gzero)
dunit = direction / norm(direction)
gunit = gzero / norm(gzero)

print(np.dot(r.gradient(), direction))
print(np.dot(gunit, dunit))
# direction = -r.gradient()
# for var in fixedVars:
#     direction[var] = 0
plotEnergy(r, -gzero, np.linspace(-20, 20, 100))

In [ ]:
print(np.dot(gzero / norm(gzero), g / norm(g)))

In [ ]:
fd_gradient_test(r, 1e-8, np.random.random_sample(direction.shape), etype=elastic_rods.EnergyType.Twist)

In [ ]:
Htrip2 = r.hessian()
Htrip2.reflectUpperTriangle()
Htrip2.rowColRemoval(fixedVars)
Htrip2.dumpBinary('Htrip2.mat')
H2 = csc_matrix(Htrip2.compressedColumn())
g2 = np.delete(g, fixedVars)
dir2 = spsolve(H2, -g2)
evalsSM, evecsSM = scipy.sparse.linalg.eigsh(H2, which='LM', sigma=-0)
evalsLM, evecsLM = scipy.sparse.linalg.eigsh(H2, which='LM')
print(evalsSM)
print(evalsLM)

In [ ]:
np.max(np.abs((H2 * np.delete(direction, fixedVars) + g2) / g2))

In [ ]:
et = elastic_rods.EnergyType.Twist
fdg = fd_gradient(r, 1e-8, et, DiffScheme.CENTERED)
relErr = np.abs((r.gradient(energyType=et) - fdg))/ np.max(np.abs(fdg))
# display(relErr)
np.max(relErr)

In [ ]:
print(fd_gradient(r, 1e-8, elastic_rods.EnergyType.Twist, DiffScheme.BACKWARD)[3 * (r.numVertices() // 2 - 1):3 * (r.numVertices() // 2 + 2)])
print(r.gradient(energyType=elastic_rods.EnergyType.Twist)[3 * (r.numVertices() // 2 - 1):3 * (r.numVertices() // 2 + 2)])
relErr[3 * (r.numVertices() // 2 - 1):3 * (r.numVertices() // 2 + 2)]

In [ ]:
Htrip2 = r.hessian()
Htrip2.reflectUpperTriangle()
Htrip2.rowColRemoval(fixedVars)
Htrip2.dumpBinary('Htrip2.mat')
H2 = csc_matrix(Htrip2.compressedColumn())
Mtrip = r.massMatrix()
Mtrip.rowColRemoval(fixedVars)
M = csc_matrix(Mtrip.compressedColumn())
evalsSM, evecsSM = scipy.sparse.linalg.eigsh(H2, which='LM', sigma=-0)
evalsLM, evecsLM = scipy.sparse.linalg.eigsh(H2, which='LM')

In [ ]:
lambdas, modes = scipy.sparse.linalg.eigsh(H2, 7, M=M, sigma=-0.001, which='LM')
full_modes = np.zeros((modes.shape[0] + len(fixedVars), modes.shape[1]))
full_modes[np.delete(np.arange(r.numDoF()), fixedVars), :] = modes

In [ ]:
lambdas

In [ ]:
import mode_viewer
modeViewer = mode_viewer.ModeViewer(r, full_modes[:], lambdas)
modeViewer.show()